# Digit Recognizer

## Goal

Identify digits from handwritten images using neural network.

## Setup

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
# Load data
train = pd.read_csv("train.csv")
#test = pd.read_csv("test.csv")

## Data Preprocessing

In [3]:
# Split data into training set and validation set
train = train.to_numpy()
split = len(train)*4//5
X_train = train[:split,1:]
y_train = train[:split,0]
X_val = train[split:,1:]
y_val = train[split:,0]

# Transform numpy array to torch tensor
X_train = torch.tensor(X_train.astype(float))
y_train = torch.tensor(y_train.astype(float))
X_val = torch.tensor(X_val.astype(float))
y_val = torch.tensor(y_val.astype(float))

# Reshape data
X_train = X_train.view(-1, 28, 28)
X_val = X_val.view(-1, 28, 28)

# Normalize data
X_train = (X_train-X_train.mean())/X_train.std()
X_val = (X_val-X_val.mean())/X_val.std()

# Cast data to float
X_train = X_train.float()
y_train = y_train.long()
X_val = X_val.float()
y_val = y_val.long()

## Model

In [4]:
class Net_2(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(7*7*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 7*7*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
class Net(nn.Module):

    def __init__(self, c=32):
        super().__init__()
        self.c = c
        self.conv1 = nn.Conv2d(1, c, kernel_size = 5, padding = 2)
        self.conv2 = nn.Conv2d(c, c*2, kernel_size = 5, padding = 2)
        self.fc1 = nn.Linear(7*7*c*2, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 7*7*self.c*2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Training

In [6]:
def training_loop(epochs, optimizer, model, loss_f, batch, x, y):
    for epoch in tqdm(range(epochs), position=0, leave=True):
        n = len(x)
        for i in range(0, n, batch):
            if i == 0:
                continue
            out = model(x[i-batch:i].unsqueeze(1))
            loss = loss_f(out, y[i-batch:i])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [7]:
model = Net()
optimizer=torch.optim.Adam(model.parameters(), lr=0.02)
training_loop(epochs=10, optimizer=optimizer, model=model, loss_f=nn.CrossEntropyLoss(), batch = 100, x = X_train, y = y_train)


100%|██████████| 10/10 [05:54<00:00, 35.41s/it]


In [8]:
correct = 0
for i in tqdm(range(len(X_val)), position = 0, leave=True):
    predict = torch.argmax(model(X_val[i].unsqueeze(0).unsqueeze(0)))
    if predict == y_val[i]:
        correct += 1
print(correct/len(X_val))

100%|██████████| 8400/8400 [00:08<00:00, 1033.66it/s]

0.1130952380952381
